<a href="https://colab.research.google.com/github/itaiassraf/IR_project/blob/main/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip  install huggingface_hub
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 5.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 8.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 5.0 MB/s 
     |████████████████████████████████| 213 kB 90.4 MB/s 
     |████████████████████████████████| 132 kB 82.0 MB/s 
     |████████████████████████████████| 127 kB 101.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [5]:
import warnings
from google.colab import drive

warnings.filterwarnings("ignore")
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from transformers import AutoTokenizer,AutoModelForMaskedLM
import datasets
train_ds=datasets.load_dataset('csv',data_files="/content/drive/MyDrive/Data/train_df.csv",sep="\t")
validaition=datasets.load_dataset('csv',data_files="/content/drive/MyDrive/Data/valid_df.csv",sep="\t")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d20b2f85b1260e47/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-748393b919aad855/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
def tokenize_function(tokenizer,dataset):
    return tokenizer(str(dataset["verse"]) )

In [ ]:
from transformers import DataCollatorForLanguageModeling,TrainingArguments

models=["Embible/distilBert-50-epochs","Embible/distilBert-20-epochs","Embible/distilBert-10-epochs","Embible/NormalBertFineTuned/10epochs"
, "Embible/NormalBertFineTuned/20epochs","Embible/NormalBertFineTuned/50epochs","Embible/tavBertFinetuned/tavBert/10epochs","Embible/tavBertFinetuned/tavBert/20epochs",
"Embible/tavBertFinetuned/tavBert/50epochs","Embible/AlephBertGimmelFineTuned/AlephBertGimmel/10epochs","Embible/AlephBertGimmelFineTuned/AlephBertGimmel/20epochs","Embible/AlephBertGimmelFineTuned/AlephBertGimmel//50epochs"]

create_model=[]
tokenize=[]
tokenization_train=[]
tokenization_validation=[]
data_collator_list=[]
for model in models:
  print(model)
  create_model.append(AutoModelForMaskedLM.from_pretrained(model, return_dict=True))
  tokenizer=AutoTokenizer.from_pretrained(model)
  tokenize.append(tokenizer)

  def tokenize_function(dataset):
    return tokenizer(str(dataset["verse"]))

  tokenization_validation.append(validaition.map(tokenize_function))
  tokenization_train.append(train_ds.map(tokenize_function))
  data_collator_list.append(DataCollatorForLanguageModeling(tokenizer=AutoTokenizer.from_pretrained(model), mlm_probability=0.15))


In [ ]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()


In [9]:
import sklearn
from datasets import load_metric
import numpy as np

metric1 = load_metric("precision")
metric2 = load_metric("recall")
metric3=load_metric('f1')
metric4=load_metric('accuracy')
lst_metrics_strings=['precision','recall','f1','accuracy']
lst_metrics=[metric1,metric2,metric3,metric4]


def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  list_metrics_results=[]
  indices = [[i for i, x in enumerate(labels[row]) if x != -100] for row in range(len(labels))]

  labels = [labels[row][indices[row]] for row in range(len(labels))]
  labels = [item for sublist in labels for item in sublist]

  predictions = [predictions[row][indices[row]] for row in range(len(predictions))]
  predictions = [item for sublist in predictions for item in sublist]
  for i,metric in enumerate(lst_metrics):
    if i==3: 
      list_metrics_results.append(metric.compute(predictions=predictions, references=labels))
      list_metrics_results[i]["eval"+'_'+lst_metrics_strings[i]] = list_metrics_results[i][lst_metrics_strings[i]]
      list_metrics_results[i].pop(lst_metrics_strings[i])
      print(list_metrics_results)
      break
    else:
      list_metrics_results.append(metric.compute(predictions=predictions, references=labels, average="weighted"))

    list_metrics_results[i]["eval"+'_'+lst_metrics_strings[i]] = list_metrics_results[i][lst_metrics_strings[i]]
    list_metrics_results[i].pop(lst_metrics_strings[i])
    
  print(0)
  return list_metrics_results

In [12]:
from transformers import Trainer
trainers=[]
results=[]
for i,model in enumerate(create_model):
  trainer = Trainer(
      model,
      train_dataset=tokenization_train[i]['train'],
      eval_dataset=tokenization_validation[i]['train'],
      data_collator=data_collator_list[i],
      tokenizer=tokenize[i],
      compute_metrics=compute_metrics,   
  )
  result=trainer.evaluate(tokenization_validation[i]['train'])
  # results.append(result)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: name, verse_idx, Unnamed: 0, verse. If name, verse_idx, Unnamed: 0, verse are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 536
  Batch size = 8
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[{'eval_precision': 0.20371410855301508}, {'eval_recall': 0.09336099585062241}, {'eval_f1': 0.09380678427268065}, {'eval_accuracy': 0.09336099585062241}]
0


TypeError: ignored

In [ ]:
trainers

In [ ]:
metric1 = load_metric("precision")
metric1.compute()